<a href="https://colab.research.google.com/github/tomhyhan/noodles/blob/main/pasta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
pwd

'/content'

In [6]:
from google.colab import userdata
from google.colab import auth

auth.authenticate_user()
token = userdata.get('token')
username = userdata.get('username')
repo_name = "noodles"

In [8]:
# !git clone https://github.com/tomhyhan/{repo_name}.git
%cd {repo_name}


/content/noodles


In [16]:
from model.utils import test_colab
test_colab()


Hello Colab! from Noodles ha


In [10]:
!git config --global user.email "tomhyhan@gmail.com"
!git config --global user.name username

In [19]:
# !git add .
# !git status
# !git commit -m "Added/Updated Colab notebook"
# !git push https://{token}@github.com/{username}/{repo_name}.git

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 362 bytes | 362.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/tomhyhan/noodles.git
   bd10370..27f027f  main -> main
